In [50]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [81]:
df = pd.read_csv('trainingdata.txt')

df.columns = ['text']
df['label'] = df['text'].apply(lambda x: x.split(' ')[0])

df['text'] = df['text'].apply(lambda x: x.split(' ', 1)[1].rsplit(' ', 2)[0])


In [80]:
df['text'][2].rsplit(' ', 2)[0]

'cobanco inc cbco year net shr cts vs dlrs net vs assets mln vs mln deposits mln vs mln loans mln vs mln note th qtr not available year includes extraordinary gain from tax carry forward of dlrs or five cts per shr'

In [84]:

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(df['text'])
X_train_counts.shape

X_train_counts

tf_transformer = TfidfTransformer(use_idf=True).fit(X_train_counts)

X_train_tfidf = tf_transformer.transform(X_train_counts)

X_train_tf.shape

clf = MultinomialNB().fit(X_train_tfidf, df['label'])

predicted = clf.predict(X_train_tfidf)

sum(predicted == df['label'])/len(df)

text_clf = Pipeline([('vect', CountVectorizer(ngram_range = (1, 2))),
                      ('tfidf', TfidfTransformer()),
                      ('clf', SGDClassifier(loss='hinge', penalty='l2',
                                            alpha=1e-6, n_iter=10, random_state=42)),
])

text_clf = text_clf.fit(df['text'][:5000], df['label'][:5000])

sum(text_clf.predict(df['text']) == df['label'])/len(df)

parameters = {'vect__ngram_range': [(1, 1), (1, 2)],
               'tfidf__use_idf': (True, False),
               'clf__alpha': (1e-4, 1e-5, 1e-6, 1e-7),
               
 }

gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1)

gs_clf = gs_clf.fit(df['text'][:5000], df['label'][:5000])

gs_clf.best_score_

0.96579999999999999

In [85]:
gs_clf.best_params_

{'clf__alpha': 1e-05, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 2)}

In [75]:
sum(text_clf.predict(df['text'][4000:]) == df['label'][4000:])/len(df[4000:])

0.96430976430976434